In [4]:
import numpy as np
import pandas as pd
import gc
pd.set_option('display.max_columns', 10)

In [2]:
pwd

'/Users/kenji/Desktop/github/Signate_try/takeda_yakuhin/notebooks'

In [89]:
train = pd.read_csv("../data/input/train.csv")
test = pd.read_csv("../data/input/test.csv")

In [90]:
score = train["Score"]
test_id = test["ID"]
del train["Score"],test["ID"],train["ID"]

In [91]:
#分散＝０
constant = []
for col in train.columns:
    if col != "Score" or col != "ID":
        if train[col].std() == 0:
            constant.append(col)
train.drop(columns=constant,axis=1,inplace=True)
test.drop(columns=constant,axis=1,inplace = True)

In [92]:
l = []
for i in train.columns.values:
    l.append(len(train[i].unique()))
ll = pd.DataFrame(np.array(l).reshape(1,-1),columns=train.columns.values)

dic = {}
for c in train.columns:
    dic[c] = ll[c].values.tolist()[0]
uniq_dict = dict(sorted(dic.items(), key=lambda x: -x[1]))
train = train[list(uniq_dict.keys())]
test = test[list(uniq_dict.keys())]

In [93]:
data = train.append(test)

In [94]:
RAW = data[data.columns[ll.values.flatten()>50]].copy()

In [30]:
def df_concat(flist):
    add_columns = flist[0].columns.values.tolist()
    add_df = flist[0].values
    for i in flist[1:]:
        add_columns += i.columns.values.tolist()
        add_df = np.c_[add_df,i.values]

    return pd.DataFrame(add_df,columns=add_columns)




In [96]:

from tqdm import tqdm
new = pd.DataFrame()
for i in tqdm(range(50,1,-1)):
    c = ll.columns[ll.values.flatten() == i]
    tmp = data[c].values

    sm = tmp.sum(axis = 1)
    sc = (tmp>0).sum(axis = 1)
    sc[sc == 0] = 1
    new[f"{i}_uniq_avg"] = sm/sc

100%|██████████| 49/49 [00:00<00:00, 49.37it/s] 


In [29]:
int_RAW = RAW[[x for x in RAW.columns if RAW[x].dtypes == int]]

In [30]:
fl_RAW= RAW[[x for x in RAW.columns if RAW[x].dtypes == float]]

In [35]:
del fl_RAW,int_RAW

In [97]:
raw = df_concat([RAW,new])

In [99]:
def add_UniqAgg(data):
    threshold = 21
    new_arr = np.zeros((data.shape[0],data.shape[1]))
    for i in tqdm(range(len(data))):
        tmp = np.unique(data.iloc[i,:])
        tmp = tmp[tmp>0]
        if len(tmp) > 0:
            new_arr[i,:len(tmp)] = tmp
            
    col = [f"FE1_{i}" for i in range(threshold)]
    new = pd.DataFrame(new_arr[:,:threshold],columns=col)
    for i in tqdm(range(len(data))):
        tmp0 = new_arr[i,:threshold]
        tmp1 = new_arr[i,threshold:]
        tmp2 = new_arr[i,:]
        
        new.loc[i,'sum_Uniq'] = tmp2[tmp2 == 0].sum()
        new.loc[i,'mean_Uniq'] = tmp1.mean()
        new.loc[i,'std_Uniq'] = tmp1.std()
        new.loc[i,'var_Uniq'] =  tmp1.var()
        new.loc[i,'max_Uniq_af'] =  tmp1.max()
        new.loc[i,'min_Uniq_af'] =  tmp1.min()
        new.loc[i,'max_Uniq_be'] =  tmp0.max()
        new.loc[i,'min_Uniq_be'] =  tmp0.min()
    
    return new
uniqagg_f= add_UniqAgg(raw)


100%|██████████| 27463/27463 [00:43<00:00, 629.14it/s]


In [100]:
def add_GreaterZeroAgg(data):
    threshold = 21
    new = pd.DataFrame()
    new_arr = np.zeros((data.shape[0],data.shape[1]))
    for i in tqdm(range(len(data))):
        tmp = data.iloc[i,:]
        tmp = tmp[tmp>0]
        if len(tmp) > 0:
            new_arr[i,:len(tmp)] = tmp
            
    col = [f"FE2_{i}" for i in range(threshold)]
    new = pd.DataFrame(new_arr[:,:threshold],columns=col)
    for i in tqdm(range(len(data))):
        tmp0 = new_arr[i,:threshold]
        tmp1 = new_arr[i,threshold:]
        tmp2 = new_arr[i,:]
        
        new.loc[i,'sum_GZ'] = tmp2[tmp2 == 0].sum()
        new.loc[i,'mean_GZ'] = tmp1.mean()
        new.loc[i,'std_GZ'] = tmp1.std()
        new.loc[i,'var_GZ'] =  tmp1.var()
        new.loc[i,'max_GZ_af'] =  tmp1.max()
        new.loc[i,'min_GZ_af'] =  tmp1.min()
        new.loc[i,'max_Gz_be'] =  tmp0.max()
        new.loc[i,'min_GZ_be'] =  tmp0.min()
    return new
GZagg_f = add_GreaterZeroAgg(raw)

100%|██████████| 27463/27463 [00:43<00:00, 636.92it/s]


In [101]:
def add_DecreaseAgg(data):
    threshold = 21
    new = pd.DataFrame()
    new_arr = np.zeros((data.shape[0],data.shape[1]))
    for i in tqdm(range(len(data))):
        tmp = data.iloc[i,:]
        tmp = tmp[tmp>0]
        _dict = dict(sorted(dict(tmp).items(), key=lambda x: -x[1]))
        tmp = tmp[list(_dict.keys())]
        if len(tmp) > 0:
            new_arr[i,:len(tmp)] = tmp
        
    col = [f"FE3_{i}" for i in range(threshold)]
    new = pd.DataFrame(new_arr[:,:threshold],columns=col)
    for i in tqdm(range(len(data))):
        tmp0 = new_arr[i,:threshold]
        tmp1 = new_arr[i,threshold:]
        tmp2 = new_arr[i,:]
        
        new.loc[i,'sum_Decrease'] = tmp2[tmp2 == 0].sum()
        new.loc[i,'mean_Decrease'] = tmp1.mean()
        new.loc[i,'std_Decrease'] = tmp1.std()
        new.loc[i,'var_Decrease'] =  tmp1.var()
        new.loc[i,'max_Decrease_af'] =  tmp1.max()
        new.loc[i,'min_Decrease_af'] =  tmp1.min()
        new.loc[i,'max_Decrease_be'] =  tmp0.max()
        new.loc[i,'min_Decrease_be'] =  tmp0.min()
    return new
Decagg_f= add_DecreaseAgg(raw)

100%|██████████| 27463/27463 [00:43<00:00, 637.61it/s]


In [102]:
def add_IncreaseAgg(data):
    threshold = 21
    new = pd.DataFrame()
    new_arr = np.zeros((data.shape[0],data.shape[1]))
    for i in tqdm(range(len(data))):
        tmp = data.iloc[i,:]
        tmp = tmp[tmp>0]
        _dict = dict(sorted(dict(tmp).items(), key=lambda x: x[1]))
        tmp = tmp[list(_dict.keys())]
        if len(tmp) > 0:
            new_arr[i,:len(tmp)] = tmp
    col = [f"FE4_{i}" for i in range(threshold)]
    new = pd.DataFrame(new_arr[:,:threshold],columns=col)
    for i in tqdm(range(len(data))):
        tmp0 = new_arr[i,:threshold]
        tmp1 = new_arr[i,threshold:]
        tmp2 = new_arr[i,:]
        
        new.loc[i,'sum_Increase'] = tmp2[tmp2 == 0].sum()
        new.loc[i,'mean_Increase'] = tmp1.mean()
        new.loc[i,'std_Increase'] = tmp1.std()
        new.loc[i,'var_Increase'] =  tmp1.var()
        new.loc[i,'max_Increase_af'] =  tmp1.max()
        new.loc[i,'min_Increase_af'] =  tmp1.min()
        new.loc[i,'max_Increase_be'] =  tmp0.max()
        new.loc[i,'min_Increase_be'] =  tmp0.min()
    return new
Incagg_f= add_IncreaseAgg(raw)

100%|██████████| 27463/27463 [00:41<00:00, 660.66it/s]


In [265]:
FE = df_concat([hist_f,dens_f])

In [266]:
FE.to_csv("../data/input/Giba_FE.csv",index = None)

In [262]:
train1,test1 = FE.iloc[:13731,:],FE.iloc[13731:,:]
train1["Score"] = score
test1["ID"] = test_id

In [252]:
del train1["Score"]

In [253]:

aa = []
for i in tqdm(train1.columns):
    if train1[i].dtype != "object":
        aa.append(i)
duplicate_f = train1[aa].T[train1[aa].T.duplicated()].index.values
train1.drop(duplicate_f,axis=1,inplace=True)
test1.drop(duplicate_f,axis=1,inplace=True)
from scipy.stats import ks_2samp
list_p_value =[]

for i in tqdm(train1.columns):
    list_p_value.append(ks_2samp(test1[i] , train1[i])[1])



  0%|          | 0/573 [00:00<?, ?it/s]

  4%|▍         | 25/573 [00:00<00:02, 243.45it/s]

  8%|▊         | 47/573 [00:00<00:02, 234.21it/s]

 12%|█▏        | 66/573 [00:00<00:02, 216.71it/s]

 15%|█▌        | 86/573 [00:00<00:02, 208.03it/s]

 20%|█▉        | 112/573 [00:00<00:02, 218.66it/s]

 24%|██▍       | 139/573 [00:00<00:01, 230.31it/s]

 29%|██▉       | 166/573 [00:00<00:01, 237.89it/s]

 33%|███▎      | 189/573 [00:00<00:01, 231.71it/s]

 37%|███▋      | 213/573 [00:00<00:01, 233.73it/s]

 42%|████▏     | 241/573 [00:01<00:01, 245.57it/s]

 46%|████▋     | 266/573 [00:01<00:01, 242.86it/s]

 51%|█████     | 291/573 [00:01<00:01, 230.94it/s]

 55%|█████▌    | 318/573 [00:01<00:01, 233.55it/s]

 61%|██████    | 347/573 [00:01<00:00, 246.91it/s]

 66%|██████▋   | 380/573 [00:01<00:00, 265.96it/s]

 73%|███████▎  | 416/573 [00:01<00:00, 288.04it/s]

 78%|███████▊  | 446/573 [00:01<00:00, 290.54it/s]

 84%|████████▍ | 483/573 [00:01<00:00, 309.36it/s]

 90%|████████▉ | 515/573 

In [258]:

Se = pd.Series(list_p_value, index = train1.columns).sort_values() 
list_discarded = list(Se[Se < .1].index)
train1.drop(list_discarded,axis=1,inplace=True)
test1.drop(list_discarded,axis=1,inplace=True)

ValueError: Length of passed values is 573, index implies 574

In [226]:
corr = set()
corr_matrix = train1.corr()
for i in tqdm(range(len(corr_matrix.columns))):
    for j in range(i):
        if abs(corr_matrix.iloc[i,j]) > 0.80:
            corr.add(corr_matrix.columns[i])
#train1.drop(corr,axis=1,inplace=True)
#test1.drop(corr,axis=1,inplace=True)



  0%|          | 0/380 [00:00<?, ?it/s]

 17%|█▋        | 64/380 [00:00<00:00, 631.68it/s]

 25%|██▌       | 96/380 [00:00<00:00, 485.01it/s]

 32%|███▏      | 120/380 [00:00<00:00, 368.80it/s]

 38%|███▊      | 143/380 [00:00<00:00, 306.12it/s]

 44%|████▎     | 166/380 [00:00<00:00, 252.32it/s]

 49%|████▉     | 188/380 [00:00<00:00, 217.65it/s]

 55%|█████▍    | 208/380 [00:00<00:00, 184.78it/s]

 59%|█████▉    | 226/380 [00:00<00:00, 163.81it/s]

 64%|██████▍   | 243/380 [00:01<00:00, 152.36it/s]

 68%|██████▊   | 259/380 [00:01<00:00, 132.76it/s]

 72%|███████▏  | 273/380 [00:01<00:00, 119.54it/s]

 75%|███████▌  | 286/380 [00:01<00:00, 110.57it/s]

 78%|███████▊  | 298/380 [00:01<00:00, 101.86it/s]

 81%|████████▏ | 309/380 [00:01<00:00, 92.97it/s] 

 84%|████████▍ | 319/380 [00:01<00:00, 87.15it/s]

 87%|████████▋ | 329/380 [00:02<00:00, 86.11it/s]

 89%|████████▉ | 338/380 [00:02<00:00, 81.41it/s]

 91%|█████████▏| 347/380 [00:02<00:00, 72.77it/s]

 93%|█████████▎| 355/380 [0

In [235]:
train1.drop(corr,axis=1,inplace=True)
test1.drop(corr,axis=1,inplace=True)

In [267]:
import json
train1,test1 = FE.iloc[:13731,:],FE.iloc[13731:,:]
train1["Score"] = score
test1["ID"] = test_id
train1.to_csv("../data/input/train_giba.csv")
test1.to_csv("../data/input/test_giba.csv")
ff = train1.columns.values
features = {"features":ff.tolist()[:-1], "target_name":"Score"}

with open("../configs/config_gibaFE.json","w") as f:
    json.dump(features,f)

In [1]:
import feather
train = feather.read_dataframe("../data/input/tr_best.feather")
test = feather.read_dataframe("../data/input/te_best.feather")
score = train.Score
del test["index"], train["Score"],train["index"], test["ID"]

In [2]:
cate_c = []
raw = train.append(test)
c = [i for i in train.columns if len(raw[i].unique()) > 2]
cc = [i for i in train.columns if len(raw[i].unique()) == 2]
num_raw = raw[c]
cate_raw= raw[cc]


In [5]:
def yao_transform(data):
    new_yao = pd.DataFrame()
    for i in data.columns:
        new_yao[f"{i}_yao"] = data[i].map(lambda x : -np.log1p(-x) if x < 0 else np.log1p(x))
    return new_yao
yeo_raw=yao_transform(num_raw)

In [25]:
import numpy as np
import pandas as pd
from tqdm import tqdm
def histgram_FE(data):
    n = 70
    new_arr = np.zeros((data.shape[0],n))

    for i in tqdm(range(len(data))):
        tmp = data.iloc[i,:]
        tmp = np.histogram(tmp,bins=n)[0]  
        new_arr[i,:] = tmp
    col = [f"histgram_{i}" for i in range(n)]
    new = pd.DataFrame(new_arr,columns=col)
    return new
hist_f = histgram_FE(yeo_raw)

100%|██████████| 27463/27463 [00:20<00:00, 1366.17it/s]


In [24]:
np.histogram(yeo_raw.iloc[0,:],bins = 70)[0]

array([  1,   0,   0,   0,   0,   0,   1,   0,   1,   1,   3,   2,   5,
         3,   3,   5, 175,  10,   9,  33,   5,  23,   4,  19,  11,  21,
        12,  11,  13,  11,   2,  11,   6,   4,   7,   8,   3,   5,   1,
         3,   2,   2,   1,   1,   0,   1,   1,   1,   0,   1,   0,   1,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   1])

In [26]:
import statsmodels.api as sm
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
def density_FE(data):
    new_arr = np.zeros((data.shape[0],50))
    for i in tqdm(range(len(data))):
        tmp = data.iloc[i,:]
        kde_model = sm.nonparametric.KDEUnivariate(tmp)
        kde_model.fit(kernel="tri",fft = False,adjust=0.25,gridsize=50,clip = [0,20])
        new_arr[i,:] = kde_model.density
    col = [f"density_{i}" for i in range(50)]
    new = pd.DataFrame(new_arr,columns=col)
    return new
dens_f = density_FE(yeo_raw)

100%|██████████| 27463/27463 [00:31<00:00, 865.46it/s]


In [46]:
hist_zero = pd.DataFrame((hist_f==0).sum(axis = 1),columns=["hist_zerosum"])
dens_zero =pd.DataFrame((dens_f==0).sum(axis = 1),columns=["dens_zerosum"])
FE = df_concat([yeo_raw,cate_raw])

In [250]:
#分散＝０
constant = []
for col in train1.columns:
    if col != "Score" or col != "ID":
        if train1[col].std() == 0:
            constant.append(col)
train1.drop(columns=constant,axis=1,inplace=True)
test1.drop(columns=constant,axis=1,inplace=True)

In [246]:
from sklearn import base
from sklearn.model_selection import KFold
class KFoldTargetEncoderTrain(base.BaseEstimator, base.TransformerMixin):

    def __init__(self, colnames,targetName,n_fold=3,k =5,verbosity=True,discardOriginal_col=False):

        self.colnames = colnames
        self.targetName = targetName
        self.n_fold = n_fold
        self.verbosity = verbosity
        self.discardOriginal_col = discardOriginal_col
        self.k = k
    def fit(self, X, y=None):
        return self


    def transform(self,X):

        assert(type(self.targetName) == str)
        assert(type(self.colnames) == str)
        assert(self.colnames in X.columns)
        assert(self.targetName in X.columns)

        mean_of_target = X[self.targetName].mean()
        target = X[self.targetName]
        kf = KFold(n_splits = self.n_fold, shuffle = False, random_state=1103)
        col_mean_name = self.colnames + '_' + 'Kfold_Target_Enc'
        X[col_mean_name] = np.nan

        for tr_ind, val_ind in kf.split(X):
            X_tr, X_val = X.iloc[tr_ind], X.iloc[val_ind]
#             print(tr_ind,val_ind)
        #X.loc[X.index[val_ind], col_mean_name] = X_val[self.colnames].map(X_tr.groupby(self.colnames)[self.targetName].mean())
            X.loc[X.index[val_ind], col_mean_name] = X_val[self.colnames].map(self.fit_smoothing(X_tr, self.colnames))
        X[col_mean_name].fillna(mean_of_target, inplace = True)

        if self.verbosity:
            encoded_feature = X[col_mean_name].values
            print('Correlation between the new feature, {} and, {} is {}.'.format(col_mean_name,
                                                                                      self.targetName,
                                                                                      np.corrcoef(X[self.targetName].values, encoded_feature)[0][1]))
        if self.discardOriginal_col:
            X = X.drop(self.targetName, axis=1)
            
        return X[col_mean_name]
    def fit_smoothing(self,df,col):
        """
        一つの変数に対するTarget_Encoding
        col : TargetEncodingしたい変数名
        """

        k = self.k
        n_i = df.groupby(col).count()[self.targetName]

        lambda_n_i = self.sigmoid(n_i, k)
        uni_map = df.groupby(col).mean()[self.targetName]

        return lambda_n_i * df.loc[:, self.targetName].mean() + (1 - lambda_n_i) * uni_map
    
    def sigmoid(self, x, k):
        return 1 / (1 + np.exp(- x / k))

class KFoldTargetEncoderTest(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self,train,colNames,encodedName):
        
        self.train = train
        self.colNames = colNames
        self.encodedName = encodedName
        
    def fit(self, X, y=None):
        return self
    def transform(self,X):
        mean =  self.train[[self.colNames,
                self.encodedName]].groupby(
                                self.colNames).mean().reset_index() 
        dd = {}
        for index, row in mean.iterrows():
            dd[row[self.colNames]] = row[self.encodedName]
        X[self.encodedName] = X[self.colNames]
        X = X.replace({self.encodedName: dd})
        return X[self.encodedName]

In [248]:
%%time
from tqdm import tqdm
def train_target_encoding(data,int_c):
    new = pd.DataFrame()
    
    for i in tqdm(int_c):
        targetc = KFoldTargetEncoderTrain(i,"Score",n_fold=3,k= 100,verbosity=False)
        new[i+'_Kfold_Target_Enc'] = targetc.fit_transform(data)
    return new
def test_target_encoding(tr,te,int_c):
    new = pd.DataFrame()
    for i in tqdm(int_c):
        test_targetc = KFoldTargetEncoderTest(tr,i,i + '_Kfold_Target_Enc')
        new[i + '_Kfold_Target_Enc'] = test_targetc.fit_transform(te)
    return new
int_c = train1.columns[:-1]
tr_tgenc_f = train_target_encoding(train1,int_c)
te_tgenc_f = test_target_encoding(train1,test1,int_c)



  0%|          | 0/290 [00:00<?, ?it/s]

  0%|          | 1/290 [00:00<04:02,  1.19it/s]

  1%|          | 2/290 [00:01<03:52,  1.24it/s]

  1%|          | 3/290 [00:02<03:48,  1.26it/s]

  1%|▏         | 4/290 [00:03<03:43,  1.28it/s]

  2%|▏         | 5/290 [00:03<03:44,  1.27it/s]

  2%|▏         | 6/290 [00:04<03:35,  1.32it/s]

  2%|▏         | 7/290 [00:05<03:31,  1.34it/s]

  3%|▎         | 8/290 [00:06<03:30,  1.34it/s]

  3%|▎         | 9/290 [00:06<03:31,  1.33it/s]

  3%|▎         | 10/290 [00:07<03:26,  1.36it/s]

  4%|▍         | 11/290 [00:08<03:23,  1.37it/s]

  4%|▍         | 12/290 [00:08<03:20,  1.39it/s]

  4%|▍         | 13/290 [00:09<03:24,  1.35it/s]

  5%|▍         | 14/290 [00:10<03:30,  1.31it/s]

  5%|▌         | 15/290 [00:11<03:38,  1.26it/s]

  6%|▌         | 16/290 [00:12<03:29,  1.31it/s]

  6%|▌         | 17/290 [00:12<03:21,  1.35it/s]

  6%|▌         | 18/290 [00:13<03:28,  1.30it/s]

  7%|▋         | 19/290 [00:14<03:34,  1.26it/s]

  7%|▋         |

CPU times: user 5min, sys: 22.9 s, total: 5min 23s
Wall time: 5min 27s


In [237]:
del train1["Score"], test1["ID"]

In [243]:
train1["Score"] = score

In [ ]:
train = pd.read_csv("../data/input/")
test = pd.read_csv("../data/input/test.csv")

In [47]:
n = len(train)
t,tt = FE.iloc[:n,:],FE.iloc[n:,:]

In [48]:
t.reset_index(inplace= True)
tt.reset_index(inplace= True)
del t["index"],tt["index"]
t.to_feather("../data/input/stack/train_yeo.feather")
tt.to_feather("../data/input/stack/test_yeo.feather")

In [45]:
FE.shape

(27463, 608)

In [55]:
from sklearn.datasets import load_iris
iris = load_iris()
x = iris.data
target = iris.target


In [54]:
f = x[:,1]
(f[1:] + f[:-1])

array([6.5, 6.2, 6.3, 6.7, 7.5, 7.3, 6.8, 6.3, 6. , 6.8, 7.1, 6.4, 6. ,
       7. , 8.4, 8.3, 7.4, 7.3, 7.6, 7.2, 7.1, 7.3, 6.9, 6.7, 6.4, 6.4,
       6.9, 6.9, 6.6, 6.3, 6.5, 7.5, 8.3, 7.3, 6.3, 6.7, 6.6, 6.1, 6.4,
       6.9, 5.8, 5.5, 6.7, 7.3, 6.8, 6.8, 7. , 6.9, 7. , 6.5, 6.4, 6.3,
       5.4, 5.1, 5.6, 6.1, 5.7, 5.3, 5.6, 4.7, 5. , 5.2, 5.1, 5.8, 6. ,
       6.1, 5.7, 4.9, 4.7, 5.7, 6. , 5.3, 5.3, 5.7, 5.9, 5.8, 5.8, 5.9,
       5.5, 5. , 4.8, 5.1, 5.4, 5.7, 6.4, 6.5, 5.4, 5.3, 5.5, 5.1, 5.6,
       5.6, 4.9, 5. , 5.7, 5.9, 5.8, 5.4, 5.3, 6.1, 6. , 5.7, 5.9, 5.9,
       6. , 5.5, 5.4, 5.4, 6.1, 6.8, 5.9, 5.7, 5.5, 5.3, 6. , 6.2, 6.8,
       6.4, 4.8, 5.4, 6. , 5.6, 5.5, 6. , 6.5, 6. , 5.8, 5.8, 5.8, 5.8,
       6.6, 6.6, 5.6, 5.4, 5.6, 6.4, 6.5, 6.1, 6.1, 6.2, 6.2, 5.8, 5.9,
       6.5, 6.3, 5.5, 5.5, 6.4, 6.4])

In [56]:
class_count = {i: len(target[target==i]) for i in np.unique(target)}
label = max(class_count.items(), key=lambda x:x[1])[0]

In [60]:
from sklearn.ensemble import AdaBoostRegressor

dict_items([(0, 50), (1, 50), (2, 50)])